# Teste pyomo

In [1]:
import numpy as np
from udgp import Instance

In [6]:
instance = Instance.artificial_molecule(5, freq=True, seed=1234567)
instance.view_input()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
instance.solve("M2")

True

In [9]:
print(instance.is_solved())
instance.view()

True


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol